In [1]:
import sys
sys.path.append("../")
from libs import *
from libs.ns_lite import *
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

Package not found.


# Lrk

In [2]:
data_path = os.path.join(DATA_PATH, 'ns_V1e-3_N5000_T50.mat')
valid_dataset = NavierStokesDatasetLite(data_path=data_path,
                                        train_data=False,)

Loading ns_V1e-3_N5000_T50.mat: start at 1743397885.05;
LOCAL RAM USAGE AT START: 0.38 GB
Loading ns_V1e-3_N5000_T50.mat: done at 1743397930.59 (45.535861 secs elapsed);
LOCAL RAM USAGE AT END: 4.49GB (+4.12GB)




In [3]:
batch_size = 2
valid_loader = DataLoader(valid_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          drop_last=False,
                          pin_memory=True)

In [4]:
# n_grid = 64
# h = 1/n_grid
# metric_func = WeightedL2Loss2d(regularizer=False, h=h)
# model = torch.load(os.path.join(MODEL_PATH, 'ns_64_4ft_196d_ln_2023-11-30.pt'))
# model.eval()
# val_metric = validate_epoch_ns(model, metric_func, valid_loader, device)
# print(f"\nBest model's validation metric in this run: {val_metric}")

# Galerkin

In [5]:
# data_path = os.path.join(DATA_PATH, 'ns_V1e-3_N5000_T50.mat')
# valid_dataset = NavierStokesDatasetLite(data_path=data_path,
#                                         train_data=False,)
# batch_size = 4

# valid_loader = DataLoader(valid_dataset,
#                           batch_size=batch_size,
#                           shuffle=False,
#                           drop_last=False,
#                           pin_memory=True)


In [6]:
# n_grid = 64
# h = 1/n_grid
# metric_func = WeightedL2Loss2d(regularizer=False, h=h)
# model = torch.load(os.path.join(MODEL_PATH, 'ns_64_4gt_196d_ln_2023-11-30.pt'))
# model.eval()
# val_metric = validate_epoch_ns(model, metric_func, valid_loader, device)
# print(f"\nBest model's validation metric in this run: {val_metric}")

# Lrk

In [7]:
# data_path = os.path.join(DATA_PATH, 'ns_V1e-3_N5000_T50.mat')
# valid_dataset = NavierStokesDatasetLite(data_path=data_path,
#                                         train_data=False,)
# batch_size = 4

# valid_loader = DataLoader(valid_dataset,
#                           batch_size=batch_size,
#                           shuffle=False,
#                           drop_last=False,
#                           pin_memory=True)


In [9]:
n_grid = 64
h = 1/n_grid
metric_func = WeightedL2Loss2d(regularizer=False, h=h)
param = torch.load(os.path.join(MODEL_PATH, 'ns_64_4Lt_196d_ln_2024-01-21.pt'))

k = len(param["v"])
print(k)

config = defaultdict(lambda: None,
                     node_feats=10+2,
                     pos_dim=2,
                     n_targets=1,
                     n_hidden=196,  # attention's d_model
                     num_feat_layers=0,
                     num_encoder_layers=4,
                     n_head=4,
                     dim_feedforward=392,
                     attention_type= None,
                     feat_extract_type=None,
                     xavier_init=0.01,
                     diagonal_weight=0.01,
                     layer_norm=True,
                     attn_norm=False,
                     return_attn_weight=True,
                     return_latent=False,
                     decoder_type='ifft',
                     freq_dim=20,  # hidden dim in the frequency domain
                     num_regressor_layers=2,  # number of spectral layers
                     fourier_modes=12,  # number of Fourier modes
                     spacial_dim=2,
                     spacial_fc=False,
                     dropout=0.0,
                     encoder_dropout=0.0,
                     decoder_dropout=0.0,
                     ffn_dropout=0.05,
                     debug=False,
                     )
config["seq_len"] = 4096
config["k"] = k
config["attention_type"] = "Lrk"
    

torch.cuda.empty_cache()
model = FourierTransformer2DLite(**config)
# print(get_num_params(model))
model = model.to(device)
model.load_state_dict(param)
model.eval()
# print(len(model.v))

val_metric = validate_epoch_ns(model, metric_func, valid_loader, device)
print(f"\nBest model's validation metric in this run: {val_metric}")

9

Best model's validation metric in this run: {'metric': 0.0023955084930639713}


In [11]:
sample = next(iter(valid_loader))
a = sample['node']
pos = sample['pos']
u = sample['target']
grid = sample['grid']
with torch.no_grad():
    model.eval()
    out_ = model(a.to(device), None, pos.to(device), grid.to(device))
    preds = out_['preds']

print(preds.shape)
print(u.shape)

torch.Size([2, 64, 64, 1])
torch.Size([2, 64, 64, 10])


In [13]:
def showcontour(z, save_path=None, show_colorbar=True, **kwargs):
    '''
    show 2D solution z of its contour
    '''
    uplot = go.Contour(z=z,
                       colorscale='RdYlBu',
                       line_smoothing=0.85,
                       line_width=0.1,
                       contours=dict(
                           coloring='heatmap',
                            #   showlabels=True,
                       ),
                       showscale=show_colorbar,
                       colorbar=dict(tickformat=".0e")
                       )
    fig = go.Figure(data=uplot,
                    layout={'xaxis': {'title': 'x-label',
                                      'visible': False,
                                      'showticklabels': False},
                            'yaxis': {'title': 'y-label',
                                      'visible': False,
                                      'showticklabels': False}},)
    # fig.update_traces(showscale=False)
    if 'template' not in kwargs.keys():
        fig.update_layout(template='plotly_dark',
                        #   margin=dict(l=0, r=0, t=0, b=0),
                        margin=dict(l=0, r=50 if show_colorbar else 0, t=0, b=0),  # 右边给 colorbar 预留空间
                        # width=600+ (100 if show_colorbar else 0),  # 右边额外扩展 100px 用于 colorbar
                        # height=600,
                        **kwargs)
    else:
        fig.update_layout(margin=dict(l=0, r=0, t=0, b=0),
                          **kwargs)
    fig.show()
    if save_path:
        save_full_path = f"{save_path}.pdf"
        fig.write_image(save_full_path, format="pdf")
        print(f"The figure have been saved: {save_path}")
        
    return fig


In [20]:
i = 1
z = preds[i, ..., 0].cpu().numpy()
z_true = u[i, ..., 0].cpu().numpy()
print(z.shape)
_ = showcontour(z, save_path="ns_pred", show_colorbar=False, width=300, height=300,)
_ = showcontour(z_true, save_path="ns_true", show_colorbar=False, width=300, height=300,)
_ = showcontour(z_true-z, save_path="ns_error", show_colorbar=True, width=400, height=300,)
print("Relative error: ", np.linalg.norm(z-z_true)/np.linalg.norm(z_true), '\n\n')

(64, 64)


The figure have been saved: ns_pred


The figure have been saved: ns_true


The figure have been saved: ns_error
Relative error:  0.0024424207 


